In [ ]:
import sys
sys.path.append('/data/ECS/PythonTutorial/')
import log
log.topic_id = '28_OrnekProjeler-Simulasyon'

# Çağrı Merkezi Simülasyonu

Bir çağrı merkezinde gelen çağrıları karşılamak için kaç agent gerektiği ve farklı agent adet alternatifleri için müşterilerin kayıp, ortalama bekleme süreleri ve agent'lar için de performansları karşılaştırılmak istenmektedir.

Bu simülasyon için bir gün içerisinde saatler bazında ortalama müşteri dağılımı için aşağıdaki oranlar verilmiştir.

* saatler = [  9,  10,  11,  12,  13,  14,  15,  16,  17]
* oranlar = [0.1, 0.2, 0.3, 0.2, 0.4, 0.3, 0.3, 0.2, 0.1]

Müşterilerin çağrı süreleri de ortalama **1-5 dk** arasında değişmektedir. **3 dk** üzerinde bekleyen müşteriler telefonu kapatmakta ve **kayıp müşteri** olarak sayılmaktadır.

Bu bilgileri kullanarak çağrı merkezinin bir gününün **09:00-18:00** saat aralığını **"gelen müşteri adeti"** ve **"agent sayısı"** bilgilerini kullanarak simüle edecek bir uygulama geliştirilmesi istenmektedir.

# Gerekli Kütüphaneler

In [ ]:
import sys
import cx_Oracle
import pandas as pd
import numpy as np
import random
import datetime as dt

# Gerekli Değişken ve Fonksiyonların oluşturulması

## Saatlik Müşteri Adetleri

Verilen oranları kullanarak k adet müşteri geldiğinde saatler bazında nasıl dağıldıklarını aşağıdaki şekilde bulabiliriz.

**random** library'sinin **choices** fonksiyonu bu ihtiyacı karşılayacak dağılımı bizlere vermektedir.

In [ ]:
saatler = [  9,  10,  11,  12,  13,  14,  15,  16,  17]
oranlar = [0.1, 0.2, 0.3, 0.2, 0.4, 0.3, 0.3, 0.2, 0.1]
customers = random.choices(saatler, oranlar, k=1000)

h,c = np.unique(customers, return_counts=True)
pd.DataFrame([dict(list(zip(h,c)))]) \
    .style \
    .background_gradient(axis=None)


In [ ]:
c

## Müşteri çağrı zamanlarının oluşturulması

Her saat için gelen çağrı adetlerin bildiğimize göre her bir saat için gelen çağrı adetlerini 0-59 dakikaları arasına **eşit olasılıkla (uniform)** dağıtabiliriz. Burada da **uniform** fonksiyonunu kullanıyoruz. Saat ve dakika bilgisini kullanarak her bir çağrının zaman değerini bulabiliriz.

In [ ]:
h_ = 9
[dt.time(hour=h_, minute=int(random.uniform(0,59)), second=0) 
 for _ in range(10)]

## Müşterilerin çağrı sürelerinin hesaplanması

Her müşterinin çağrı sürelerinin 1-5 dk arasında olduğunu biliyorduk. Bu bilgiyi kullanarak her çağrının geliş zamanı ve rastgele süresini hesaplayarak **müşteriler** değişkeninde saklıyoruz.

In [ ]:
musteriler = [
    (dt.time(hour=h_, 
             minute=int(random.uniform(0,59)), 
             second=0
            ),
     random.randint(1,5)
    )
    for h_,c_ in zip(h,c)
    for i in range(c_)
]

musteriler = pd.DataFrame(sorted(musteriler))
musteriler

## Müşteri Oluştur fonksiyonunun oluşturulması

Şu ana kadar yaptığımız işlemleri tek bir fonksiyon altında toplayarak bir gün için random bir gelen çağrı listesi oluşturabiliriz.

In [ ]:
def musteri_olustur(k=1000):
    # önce saatler bazında gelecek müşteri çağrılarının adetini buluyoruz
    population = [  9,  10,  11,  12,  13,  14,  15,  16,  17]
    weights =    [0.1, 0.2, 0.3, 0.2, 0.4, 0.3, 0.3, 0.2, 0.1]
    customers = random.choices(population, weights, k=k)

    h,c = np.unique(customers, return_counts=True)
    pd.DataFrame([dict(list(zip(h,c)))]).style.background_gradient(axis=None)

    # ardından bu dağılıma uyacak şekilde müşterileri random dadikalara dağıtıyoruz
    musteriler = [
        (dt.time(hour=h_, 
                 minute=int(random.uniform(0,59)), 
                 second=0
                ),
         random.randint(1,5)
        )
        for h_,c_ in zip(h,c)
        for i in range(c_)
    ]

    musteriler = pd.DataFrame(sorted(musteriler))
    return musteriler

omusteriler = musteri_olustur(k=5000)
omusteriler

## Simülasyonun oluşturulması

Hesapladığımız gelen çağrılar içerisinde bir döngü ile her bir müşterinin bir agent tarafından cevaplanması ve çağrı sonunda bekleyen bir sonraki müşterinin alınması şeklinde bir döngü izliyor olacağız.

Döngü adımlarını aşağıdaki gibi sıralayabiliriz:
* öncelikle döngü içerisinde kullanılacak değişkenleri tanımlıyoruz:
    * t: döngü zamanı (başlangıç 09:00)
    * agents: agent'lara ait bilgileri tutacak bir değişken
    * max_bekleme: müşteri max bekleme süresi
    * ct: bekleme süresini aşan müşteri adeti
    * num_agents: agent sayısı
    
İlave olarak saat değerine dakika eklemek için de küçük bir fonksiyon oluşturuyoruz. Bu işlemin amacı, tekrarlayan bu işlemi daha az kod yazarak hesaplayabilmektir.

In [ ]:
musteriler = omusteriler.copy()
display(musteriler.head(10))

# mesai başlangıç
t = dt.time(9,0)

# çalışan agent sayısı
num_agents = 8

# agent'ları tutarn değişken
agents = {i:{'status':'Bos', 
             'customers':[]
            } 
          for i in range(num_agents)}

# müşteri max bekleme süresi
max_bekleme = 3
# bekleme süresini aşan müşteriler
ct = 0

def bitis_zamani_hesapla(baslangic, sure):
    bitis_zamani = (dt.datetime.combine(dt.datetime.now(), baslangic) + dt.timedelta(minutes= int(sure))).time()
    return bitis_zamani

## Simülasyon Döngüsü

Değişkenler ve fonksiyonlar tanımlandığına göre artık simülasyonumuzu mesaiye başlatabiliriz.
* Öncelikle, her bir agent'ı kontrol edip statü durumu **Dolu** olup işlemi tamamlanan müşteri varsa o müşteriyi agent'tan çıkartıyoruz. 
* Ardından yine statüsü **Boş** olan her bir agent için **beleyen müşteri** varsa ilk sıradaki müşteriyi agent'a atıyoruz ve müşteriyi **musteriler** datasından çıkartıyoruz ve agent'ın statüsünü **Dolu** olarak güncelliyoruz.
* Daha sonra döngünün **t** anı itibarı ile bekleme listesinde yer alıp **max_bekleme** süresini aşan müşteriler varsa bu müşterileri **kayıp müşteri** olarak listeden çıkartıyoruz ve **ct** sayacını bir artırıyoruz.
* Bu işlemin ardından bekleme listesinde müşteri kalmamışsa 
    * döngüden çıkıyoruz. **break**
* Bekleyen müşteri varsa 
    * bir sonraki müşterinin geliş saatini buluyoruz.
    * agent'larda işlemi devam eden müşterilerin bitiş saatlerini buluyoruz
    * boş durumdaki agent'lar için bir sonraki müşterinin geliş saatini yazıyoruz
    * agent'lardaki en küçük bitiş saatini (boş olanlar için sonraki müşteri geliş saati) bularak döngünün bir sonraki adımdaki **"t"** döngü saatini belirliyoruz. **min(bitis_zamanlari)**
    * döngüye devam ediyoruz

In [ ]:
i = 0

while True:
    # her bir agent'ın durumunu kontrol et
    print(t)
    
    # eger agent'da suresi dolan işlem varsa cagriyi kapat
    for a in agents:
        if agents[a]['status']=='Dolu':
            agent_son_musteri_id = agents[a]['customers'][-1][0]
            son_musteri_baslangic = agents[a]['customers'][-1][3]
            son_musteri_islemsure = agents[a]['customers'][-1][2]
            son_musteri_islembitis = bitis_zamani_hesapla(son_musteri_baslangic, son_musteri_islemsure)
            
            # eğer işlem bitmişse agent durumunu güncelle
            if son_musteri_islembitis <= t:
                agents[a]['status']='Bos'
                print(f'a:{a:<5}s:{"-":<5}c:{agent_son_musteri_id}')
                
    
    # eger agent'da devam eden işlem yoksa VE t zamanında bekleyen müşteri varsa yeni bir müşteri ekle
    for a in agents:
        if agents[a]['status']=='Bos' and len(musteriler[musteriler.iloc[:,0]<=t])>0:
            
            # sıradaki müşteriyi al
            musteri = (musteriler.index[0], musteriler.iloc[0,0], musteriler.iloc[0,1], t)
            
            # müşteriyi listeden düşür
            musteriler.drop(musteriler.index[0], inplace=True)

            # müşteriyi agent'a ekle, statüsünü güncelle
            agents[a]['customers'].append(musteri)
            agents[a]['status']='Dolu'
            print(f'a:{a:<5}s:{"+":<5}c:{musteri}')

            
    # henüz işlem saati gelmemiş müşterilerden bekleme süresi dolan müşterileri sıradan çıkart
    for i,c in musteriler[musteriler.iloc[:,0]<t].iterrows():
        if bitis_zamani_hesapla(c[0], max_bekleme)<=t:
            musteriler.drop(i, inplace=True)
            ct = ct + 1
            print(f'c:{i:<5}s:{"-":<5}')
            
    #display(agents)
    
    # eğer müşteri kalmadıysa döngüden çık
    if len(musteriler)==0:
        break
    
    # bir sonraki gelecek müşterinin çağrı saati
    sonraki_musteri_zamani = min(musteriler.iloc[:,0])
    
    # minimum t zamanını bul
    # agent'ların içerisinde işlemi ilk bitecek olanın bitiş zamanı
    # eğer bir agent'ın müşterisi yoksa, onun bitiş zamanını ilk gelecek müşteri olarak belirle
    bitis_zamanlari = [bitis_zamani_hesapla(agents[a]['customers'][-1][3], agents[a]['customers'][-1][2])
                       if agents[a]['status']=='Dolu' and len(agents[a]['customers'])>0
                       else sonraki_musteri_zamani
                       for a in agents
                      ]
    
    t = min(bitis_zamanlari)


## Performans Ölçümü

Döngümüz tüm müşteriler işlem görüp tamamlandıktan sonra performans ölçümü için **"agents"** değişkenindeki bilgileri kullanabiliriz.

* Öncelikle agent'a yeni bir müşteri eklerken kullandığımız **musteri** değişkeninde müşterinin id'si, geliş saati, işlem süresi ve agent'ın müşteriyi cevaplama saati bilgilerini kullanarak her müşterinin **"bekleme süresi"**'ni buluyoruz.
* İlk metriğimiz **"Ortalama Bekleme Süresi"** bu sürelerin ortalamasını göstermektedir.
* **"Kayıp Müşteri Adet"** metriği ise direk olarak döngüde süresi dolan müşterileri sayarken kullandığımız **ct** değişkeni.
* Ardından agent'ların performanslarını ölçmek için her bir agent'ın hizmet verdiği müşteri adeti, müşteri işlem süresi ve bu işlem süresinin 9 saatlik mesai içerisinde oranı'nı buluyoruz.
* Sonrasında da her bir agent için bu değerleri ve agent'lar toplamını ekrana yazdırıyoruz.

In [ ]:
agents

In [ ]:
sureler = [(dt.datetime.combine(dt.datetime.now(), c[3])-
            dt.datetime.combine(dt.datetime.now(), c[1])).seconds/60
           for a in agents
           for c in agents[a]['customers']
          ]

print(f'{"Ort. Bekleme Süresi":<30}:{np.mean(sureler):.2f}')
#print(f'{"Kayıp Müşteri Adet ":<30}:{ct:.0f}, {ct/musteri_sayisi*100:.1f}%')
p = [
        np.vstack([([1, c[2], c[2]/540]) 
                   for c in agents[a]['customers']
                  ]).sum(axis=0)
        for a in agents
    ]

print('Agent performans:')
for i, ap in enumerate(p):
    print(f'{i:>30}: adet:{ap[0]:>5.0f}, sure:{ap[1]:>5.0f} dk,  ft:{ap[2]:>6.2f}')
print(f'{"Toplam":>30}: adet:{np.array(p)[:,0].sum():>5.0f}, sure:{np.array(p)[:,1].sum():>5.0f} dk,  ft:{np.array(p)[:,1].sum()/(len(p)*540):>6.2f}')

In [ ]:
len()

# Class'a Dönüştür

Şimdi yaptığımız tüm bu işlemleri bir class'a dönüştürelim


In [ ]:
import sys
import cx_Oracle
import pandas as pd
import numpy as np
import random
import datetime as dt

class CSSimulasyon():
    
    def __init__(self, musteri_sayisi=1000, agent_sayisi=3, max_bekleme_suresi=3):
        print('Gişe simülasyon başlatıldı')
        self.degiskenleri_tanimla(musteri_sayisi=musteri_sayisi,
                                  agent_sayisi=agent_sayisi, 
                                  max_bekleme_suresi=max_bekleme_suresi)
        print(f'Değişkenler tanımlandı')
        print(f'Müşteri adet:{musteri_sayisi}, Agent adet:{agent_sayisi}')
        self.musteriler = self.musteri_olustur(musteri_sayisi)
        print(f'Müşteri datası oluşturuldu')
        #print([d for d in dir(gs) if d[0]!='_'])
        

    def degiskenleri_tanimla(self, musteri_sayisi, agent_sayisi, max_bekleme_suresi):
        # mesai başlangıç
        self.t = dt.time(9,0)
        
        # musteri sayısı
        self.musteri_sayisi = musteri_sayisi
        
        # çalışan agent sayısı
        self.agent_sayisi = agent_sayisi

        # müşteri max bekleme süresi
        self.max_bekleme = max_bekleme_suresi
        # bekleme süresini aşan müşteriler
        self.ct = 0
        
    def bitis_zamani_hesapla(self, baslangic, sure):
        bitis_zamani = (dt.datetime.combine(dt.datetime.now(), baslangic) + dt.timedelta(minutes= int(sure))).time()
        return bitis_zamani
    
    def musteri_olustur(self, k=1000):
        # önce saatler bazında gelecek müşteri çağrılarının adetini buluyoruz
        population = [  9,  10,  11,  12,  13,  14,  15,  16,  17]
        weights =    [0.1, 0.2, 0.3, 0.2, 0.4, 0.3, 0.3, 0.2, 0.1]
        customers = random.choices(population, weights, k=k)

        h,c = np.unique(customers, return_counts=True)
        pd.DataFrame([dict(list(zip(h,c)))]).style.background_gradient(axis=None)

        # ardından bu dağılıma uyacak şekilde müşterileri random dadikalara dağıtıyoruz
        musteriler = [
            (dt.time(hour=h_, 
                     minute=int(random.uniform(0,59)), 
                     second=0
                    ),
             random.randint(1,5)
            )
            for h_,c_ in zip(h,c)
            for i in range(c_)
        ]

        self.musteriler = pd.DataFrame(sorted(musteriler))
        return self.musteriler    
    
    def simulate(self, yeni_musteri_datasi=False, debug=False):
        
        # ------------------------------------------------------------------
        # yeni simülasyonda resetlenecek değişkenler 
        #
        
        self.ct = 0
        
        # agent'ları tutarn değişken
        self.agents = {i:{'status':'Bos', 
                     'customers':[]
                    } 
                  for i in range(self.agent_sayisi)}

        
        # parametreye göre yeni müşteri datası gerekiyorsa oluştur
        if yeni_musteri_datasi:
            musteriler = self.musteri_olustur(self.musteri_sayisi).copy()
        else:
            musteriler = self.musteriler.copy()
        
        t = self.t
        
        # ------------------------------------------------------------------
        
        i = 0
        while True:
            # her bir agent'ın durumunu kontrol et
            if debug: print(t)

            # eger agent'da suresi dolan işlem varsa cagriyi kapat
            agents = self.agents
            for a in agents:
                if agents[a]['status']=='Dolu':
                    agent_son_musteri_id = agents[a]['customers'][-1][0]
                    son_musteri_baslangic = agents[a]['customers'][-1][3]
                    son_musteri_islemsure = agents[a]['customers'][-1][2]
                    son_musteri_islembitis = self.bitis_zamani_hesapla(son_musteri_baslangic, son_musteri_islemsure)

                    # eğer işlem bitmişse agent durumunu güncelle
                    if son_musteri_islembitis <= t:
                        agents[a]['status']='Bos'
                        if debug: print(f'a:{a:<5}s:{"-":<5}c:{agent_son_musteri_id}')


            # boştaki agent'a çağrı atadığımız için agetn'ları karıştırıyoruz
            agent_keys = list(agents.keys())
            random.shuffle(agent_keys)
            
            # eger agent'da devam eden işlem yoksa VE t zamanında bekleyen müşteri varsa yeni bir müşteri ekle
            for a in agent_keys:
                if agents[a]['status']=='Bos' and len(musteriler[musteriler.iloc[:,0]<=t])>0:

                    # sıradaki müşteriyi al
                    musteri = (musteriler.index[0], musteriler.iloc[0,0], musteriler.iloc[0,1], t)

                    # müşteriyi listeden düşür
                    musteriler.drop(musteriler.index[0], inplace=True)

                    # müşteriyi agent'a ekle, statüsünü güncelle
                    agents[a]['customers'].append(musteri)
                    agents[a]['status']='Dolu'
                    if debug: print(f'a:{a:<5}s:{"+":<5}c:{musteri}')


            # henüz işlem saati gelmemiş müşterilerden bekleme süresi dolan müşterileri sıradan çıkart
            for i,c in musteriler[musteriler.iloc[:,0]<t].iterrows():
                if self.bitis_zamani_hesapla(c[0], self.max_bekleme)<=t:
                    musteriler.drop(i, inplace=True)
                    self.ct = self.ct + 1
                    if debug: print(f'c:{i:<5}s:{"-":<5}')

            #display(agents)

            # eğer müşteri kalmadıysa döngüden çık
            if len(musteriler)==0:
                break

            # bir sonraki gelecek müşterinin çağrı saati
            sonraki_musteri_zamani = min(musteriler.iloc[:,0])

            # minimum t zamanını bul
            # agent'ların içerisinde işlemi ilk bitecek olanın bitiş zamanı
            # eğer bir agent'ın müşterisi yoksa, onun bitiş zamanını ilk gelecek müşteri olarak belirle
            bitis_zamanlari = [self.bitis_zamani_hesapla(agents[a]['customers'][-1][3], agents[a]['customers'][-1][2])
                               if agents[a]['status']=='Dolu' and len(agents[a]['customers'])>0
                               else sonraki_musteri_zamani
                               for a in agents
                              ]

            t = min(bitis_zamanlari)
        
        self.agents = agents
        self.sonuclar()
    
    def sonuclar(self):
        agents = self.agents
        
        sureler = [(dt.datetime.combine(dt.datetime.now(), c[3])-
                    dt.datetime.combine(dt.datetime.now(), c[1])).seconds/60
                   for a in agents
                   for c in agents[a]['customers']
                  ]

        print(f'{"Ort. Bekleme Süresi":<30}:{np.mean(sureler):.2f}')
        print(f'{"Kayıp Müşteri Adet ":<30}:{self.ct:.0f}, {self.ct/self.musteri_sayisi*100:.1f}%')
        p = [
                np.vstack([([1, c[2], c[2]/540]) 
                           for c in agents[a]['customers']
                          ]).sum(axis=0)
                for a in agents
            ]
        p
        print('Agent performans:')
        for i, ap in enumerate(p):
            print(f'{i:>30}: '+
                  f'adet:{ap[0]:>5.0f}, '+
                  f'sure:{ap[1]:>5.0f} dk,  '+
                  f'ft:{ap[2]:>6.2f}')
        print(f'{"Toplam":>30}: '+
              f'adet:{np.array(p)[:,0].sum():>5.0f}, '+
              f'sure:{np.array(p)[:,1].sum():>5.0f} dk,  '+
              f'ft:{np.array(p)[:,1].sum()/(len(p)*540):>6.2f} '+u"\u00B1"+f'{np.std(np.array(p)[:,2]):.2f}'
             )


In [ ]:
gs = CSSimulasyon(musteri_sayisi=1000)

In [ ]:
gs.agent_sayisi=3
gs.simulate()

In [ ]:
gs.agent_sayisi=5
gs.simulate()

In [ ]:
gs.agent_sayisi=8
gs.simulate()

In [ ]:
gs.agent_sayisi=20
gs.simulate()